# 使用 Mistral 模型構建

## 介紹

本課程將涵蓋：
- 探索不同的 Mistral 模型
- 了解每個模型的使用案例和場景
- 代碼範例展示每個模型的獨特功能。


## Mistral 模型

在本課程中，我們將探索 3 種不同的 Mistral 模型：
**Mistral Large**、**Mistral Small** 和 **Mistral Nemo**。

這些模型皆可在 Github 模型市集中免費取得。本筆記本中的程式碼將使用這些模型來執行。以下是使用 Github 模型進行[AI 模型原型設計](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst)的更多詳細資訊。


## Mistral Large 2 (2407)
Mistral Large 2 目前是 Mistral 的旗艦模型，專為企業使用而設計。

該模型是對原始 Mistral Large 的升級，提供
- 更大的上下文視窗 - 128k 對比 32k
- 在數學和程式設計任務上的更佳表現 - 平均準確率 76.9% 對比 60.4%
- 提升的多語言表現 - 語言包括：英語、法語、德語、西班牙語、義大利語、葡萄牙語、荷蘭語、俄語、中文、日語、韓語、阿拉伯語和印地語。

憑藉這些特點，Mistral Large 擅長於
- *檢索增強生成 (RAG)* - 由於更大的上下文視窗
- *函數調用* - 該模型具有原生函數調用功能，允許與外部工具和 API 集成。這些調用可以並行進行，也可以按順序依次執行。
- *程式碼生成* - 該模型在 Python、Java、TypeScript 和 C++ 生成方面表現優異。


### 使用 Mistral Large 2 的 RAG 範例


在此範例中，我們使用 Mistral Large 2 來對文本文件執行 RAG 模式。問題以韓文撰寫，詢問作者在大學前的活動。

它使用 Cohere Embeddings 模型來建立文本文件及問題的嵌入向量。此範例中，使用 faiss Python 套件作為向量庫。

傳送給 Mistral 模型的提示包含問題以及與問題相似的檢索片段。模型隨後提供自然語言回應。


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Small 是 Mistral 系列模型中屬於 premier/enterprise 類別的另一款模型。如其名稱所示，這款模型是一個小型語言模型（SLM）。使用 Mistral Small 的優點包括： 
- 相較於 Mistral LLMs（如 Mistral Large 和 NeMo）節省成本 - 價格降低 80%
- 低延遲 - 相較於 Mistral 的 LLMs 回應更快
- 靈活 - 可在不同環境中部署，對所需資源的限制較少。 

Mistral Small 非常適合： 
- 文字相關任務，如摘要、情感分析和翻譯。 
- 由於成本效益高，適合頻繁請求的應用 
- 低延遲的程式碼任務，如程式碼審查和建議


## 比較 Mistral Small 與 Mistral Large

要顯示 Mistral Small 與 Large 之間的延遲差異，請執行以下儲存格。

您應該會看到回應時間相差約 3-5 秒。也請注意相同提示下的回應長度和風格。


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

與本課程中討論的其他兩個模型相比，Mistral NeMo 是唯一擁有 Apache2 授權的免費模型。

它被視為早期 Mistral 開源大型語言模型 Mistral 7B 的升級版。

NeMo 模型的其他一些特點包括：

- *更高效的分詞：* 此模型使用 Tekken 分詞器，而非較常用的 tiktoken。這使其在多語言和程式碼上的表現更佳。

- *微調：* 基礎模型可用於微調。這為需要微調的使用案例提供了更大的靈活性。

- *原生函數調用* - 與 Mistral Large 類似，此模型已經過函數調用訓練。這使其成為首批具備此功能的開源模型之一。


## Mistral NeMo

與本課程中討論的其他兩個模型相比，Mistral NeMo 是唯一具有 Apache2 許可證的免費模型。

它被視為早期 Mistral 開源大型語言模型 Mistral 7B 的升級版。

NeMo 模型的其他一些特點包括：

- *更高效的分詞：* 此模型使用 Tekken 分詞器，而非更常用的 tiktoken。這使其在多語言和程式碼處理上表現更佳。

- *微調：* 基礎模型可用於微調。這為需要微調的使用案例提供了更大的靈活性。

- *原生函數調用* - 與 Mistral Large 類似，此模型經過函數調用訓練。這使其成為首批開源模型中獨特的一員。


### 比較分詞器

在此範例中，我們將比較 Mistral NeMo 與 Mistral Large 在分詞處理上的差異。

兩個範例都使用相同的提示詞，但你應該會看到 NeMo 回傳的標記數量比 Mistral Large 少。


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## 學習不止於此，繼續旅程

完成本課程後，請查看我們的[生成式 AI 學習合集](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst)，持續提升您的生成式 AI 知識！


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責聲明**：  
本文件係使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。雖然我們致力於確保翻譯的準確性，但請注意，自動翻譯可能包含錯誤或不準確之處。原始文件的母語版本應視為權威來源。對於重要資訊，建議採用專業人工翻譯。我們不對因使用本翻譯而產生的任何誤解或誤釋負責。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
